<a href="https://colab.research.google.com/github/marcmontb/data_driven-VC/blob/main/Neural_Transfer_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Setup
First, ensure you have the required libraries installed. You can install them using the following commands:

In [7]:
!pip install torch torchvision pillow matplotlib

# Step 2: Import Libraries
Create a new cell in your Jupyter notebook and import the necessary libraries:

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NameError: name '_C' is not defined

Import content and style images

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 1-style.jpg to 1-style.jpg
Saving 1-content.jpg to 1-content.jpg


# Step 3: Image Preprocessing
We define a function to load and preprocess images. The preprocessing involves resizing the image, converting it to a tensor, and normalizing it with mean and standard deviation values from the ImageNet dataset.

In [4]:
import numpy as np  # Import NumPy

def image_loader(image_path, max_size=400, shape=None):
    image = Image.open(image_path).convert('RGB')

    # Resize the image
    size = max_size if max(image.size) > max_size else max(image.size)
    if shape is not None:
        size = shape

    transform = transforms.Compose([
        transforms.Resize(size),  # Resize the image to the desired size
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image using ImageNet mean and std
    ])

    image = transform(image).unsqueeze(0)  # Add a batch dimension
    return image.to(device, torch.float)

# Load and preprocess the content and style images
content_image = image_loader('path_to_content_image.jpg')
style_image = image_loader('path_to_style_image.jpg', shape=[content_image.size(2), content_image.size(3)])

# Display the images
def imshow(tensor, title=None):
    image = tensor.cpu().clone().squeeze(0).detach().numpy()  # Convert tensor to numpy array
    image = image.transpose(1, 2, 0)  # Transpose dimensions to HWC (height, width, channels)
    mean = np.array([0.485, 0.456, 0.406])  # ImageNet mean
    std = np.array([0.229, 0.224, 0.225])  # ImageNet std
    image = std * image + mean  # De-normalize the image
    image = np.clip(image, 0, 1)  # Clip values to the range [0, 1]

    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.show()

imshow(content_image, title='Content Image')
imshow(style_image, title='Style Image')



NameError: name 'Image' is not defined

# Step 4: Define Loss Functions
We define the content loss and style loss functions.
The Gram matrix is used to measure style loss.

In [ ]:
class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        self.target = target.detach()  # Detach target to prevent gradient computation

    def forward(self, input):
        self.loss = nn.functional.mse_loss(input, self.target)  # Compute mean squared error loss
        return input

def gram_matrix(input):
    a, b, c, d = input.size()  # Batch size, number of feature maps, height, width
    features = input.view(a * b, c * d)  # Reshape input to 2D tensor
    G = torch.mm(features, features.t())  # Compute Gram matrix
    return G.div(a * b * c * d)  # Normalize Gram matrix

class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()  # Detach target Gram matrix

    def forward(self, input):
        G = gram_matrix(input)  # Compute Gram matrix for input
        self.loss = nn.functional.mse_loss(G, self.target)  # Compute mean squared error loss
        return input

# Step 5: Build the Model
We build the model using the VGG19 network and add content and style loss layers at the appropriate points.

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

content_layers = ['conv_4']  # Layers used for content representation
style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']  # Layers used for style representation

class StyleTransferModel(nn.Module):
    def __init__(self, cnn, content_img, style_img, content_layers, style_layers):
        super(StyleTransferModel, self).__init__()
        self.content_losses = []
        self.style_losses = []
        self.model = nn.Sequential()

        i = 0
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = f'conv_{i}'
            elif isinstance(layer, nn.ReLU):
                name = f'relu_{i}'
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = f'pool_{i}'
            elif isinstance(layer, nn.BatchNorm2d):
                name = f'bn_{i}'

            self.model.add_module(name, layer)

            if name in content_layers:
                target = self.model(content_img).detach()
                content_loss = ContentLoss(target)
                self.model.add_module(f'content_loss_{i}', content_loss)
                self.content_losses.append(content_loss)

            if name in style_layers:
                target_feature = self.model(style_img).detach()
                style_loss = StyleLoss(target_feature)
                self.model.add_module(f'style_loss_{i}', style_loss)
                self.style_losses.append(style_loss)

    def forward(self, x):
        self.model(x)
        content_loss = sum(cl.loss for cl in self.content_losses)
        style_loss = sum(sl.loss for sl in self.style_losses)
        return content_loss, style_loss

# Initialize the generated image
input_img = content_image.clone()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:08<00:00, 68.9MB/s]


# Step 6: Define the Optimizer
We define the optimizer (LBFGS)  to update the generated image.



In [ ]:
optimizer = optim.LBFGS([input_img.requires_grad_()])


# Step 7: Perform Style Transfer
We perform the style transfer by iteratively minimizing the total loss, which is a weighted sum of content and style losses, and display the final image.


In [ ]:
style_transfer_model = StyleTransferModel(cnn, content_image, style_image, content_layers, style_layers).to(device)

num_steps = 300
style_weight = 1e6
content_weight = 1

for step in range(num_steps):
    def closure():
        input_img.data.clamp_(0, 1)  # Clamp image values to [0, 1]
        optimizer.zero_grad()  # Zero the gradients
        content_loss, style_loss = style_transfer_model(input_img)  # Forward pass
        loss = content_weight * content_loss + style_weight * style_loss  # Compute total loss
        loss.backward()  # Backward pass
        return loss

    optimizer.step(closure)  # Optimize

    input_img.data.clamp_(0, 1)  # Clamp image values to [0, 1]

# Display the final generated image
imshow(input_img, title='Generated Image')

